In [1]:
from keras.layers import Input, Lambda, Dense, Flatten , GlobalAveragePooling2D , Dropout
from keras.models import Model
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
import numpy as np
from glob import glob
import matplotlib.pyplot as plt
import pandas as pd
from tensorflow.keras.callbacks import TensorBoard
import time
from datetime import datetime

Using TensorFlow backend.


In [2]:
logdir = "logs\\scalars\\" + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = TensorBoard(log_dir=logdir)

In [3]:
IMAGE_SIZE = [224, 224]

In [4]:
train_path = 'Dataset/train'
valid_path = 'Dataset/test'

In [5]:
folders = glob('Dataset/train/*')
print(folders)

['Dataset/train\\NORMAL', 'Dataset/train\\PNEUMONIA']


In [6]:
vgg = VGG16(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

In [7]:
x=Flatten()(vgg.output)
# x=Dense(1024,activation='relu')(x)
# x=Dropout(0.3)(x)
# x=Dense(512,activation='relu')(x)
# x=Dropout(0.3)(x)
prediction = Dense(len(folders), activation='softmax')(x)

In [8]:
model=Model(inputs=vgg.input,outputs=prediction)

In [9]:
# model.trainable = True

# set_trainable = False

# for layer in model.layers:
#     if layer.name in ['block5_conv1', 'block4_conv1']:
#         set_trainable = True
#     if set_trainable:
#         layer.trainable = True
#     else:
#         layer.trainable = False

In [10]:
for layer in vgg.layers:
    layer.trainable = False

In [11]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

In [12]:
model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)

In [13]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

training_set = train_datagen.flow_from_directory('Dataset/train',
                                                 target_size = (224, 224),
                                                 batch_size = 64,
                                                 class_mode = 'categorical')

test_set = test_datagen.flow_from_directory('Dataset/test',
                                            target_size = (224, 224),
                                            batch_size = 64,
                                            class_mode = 'categorical')

Found 2682 images belonging to 2 classes.
Found 624 images belonging to 2 classes.


In [ ]:
r = model.fit_generator(
  training_set,
  validation_data=test_set,
  epochs=6,
  steps_per_epoch=len(training_set),
  validation_steps=len(test_set),
    callbacks=[tensorboard_callback]
)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Epoch 1/6
11/42 [======>.......................] - ETA: 10:19 - loss: 0.8335 - acc: 0.6719

In [ ]:
plt.plot(r.history['loss'], label='train loss')
plt.plot(r.history['val_loss'], label='val loss')
plt.legend()
plt.show()
plt.savefig('LossVal_loss')

# accuracies
plt.plot(r.history['acc'], label='train acc')
plt.plot(r.history['val_acc'], label='val acc')
plt.legend()
plt.show()
plt.savefig('AccVal_acc')

In [ ]:
# model.save("covid19_vgg19.h5")

In [ ]:
# ############################## DATA AUGMENTATION ############################################################

# from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
# import os

# datagen = ImageDataGenerator(
#         rescale=1./255,
#         shear_range=0.2,
#         zoom_range=0.2,
#         horizontal_flip=True)

# path = "Dataset/train/PNEUMONIA"
# files = os.listdir(path)
# total = len(files)
# count = 0
# for file in files:
#     new_path = os.path.join(path,file)
#     img = load_img(new_path)  # this is a PIL image
#     x = img_to_array(img)  # this is a Numpy array with shape (3, 150, 150)
#     x = x.reshape((1,) + x.shape)  # this is a Numpy array with shape (1, 3, 150, 150)

#     # the .flow() command below generates batches of randomly transformed images
#     # and saves the results to the `preview/` directory
#     i = 0
#     count+=1
#     for batch in datagen.flow(x, batch_size=1,
#                               save_to_dir=path, save_prefix=file, save_format='jpeg'):
#         i += 1
#         print("{} images of {} no image is created . ".format( str(i) , str(count) ) )
#         if i > 4:
#             break  # otherwise the generator would loop indefinitely
#     print("{} images out of {} have been augmented".format( str(count) , str(total) ) )
#     print("----------------------------------------------")